Начнём с импортирования библиотек

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

прописываем параметры подключения

In [ ]:
import pymysql.cursors  
 
# Подключиться к базе данных.
connection = pymysql.connect(host='185.241.192.198',
                             user='user',
                             password='5QnDCLfgsVXfmdUX',                             
                             db='soccer',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

Пишем наш первый запрос и проверяем, смогли ли мы подключиться

In [ ]:
with connection.cursor() as cursor:
       
        # SQL 
        sql = "SELECT * FROM country limit 5 "
         
        # Выполнить команду запроса (Execute Query).
        cursor.execute(sql)
         
        print ("cursor.description: ", cursor.description)
 
        #print()

In [ ]:
df = pd.read_sql('SELECT player.player_api_id, player.player_name,avg(p_a.overall_rating) as avg_r,row_number() over( order by avg(p_a.overall_rating) desc) as rating FROM player JOIN player_attributes as p_a ON player.player_api_id =  p_a.player_api_id GROUP BY player.player_api_id, player.player_name', con= connection)

In [ ]:
df

In [ ]:
data = pd.read_csv("C:/Users/s.voronkov/Downloads/student-mat.csv")
data.head()

Совершаем первые простейшие итерации
Начнём с подсчёта строк и столбцов с помощью команды shape

In [ ]:
data.shape

проверим, есть ли пропуски в данных:

In [ ]:
data.info()

Пропуски - частое явление при анализе данных, они часто препятствуют совершению некоторых методов классификации или построению графиков. Есть несколько методов работы с ними:
1. Замена на модальное значиение
2. Замена на среднее значение
3. Замена на медианное значение
4. Удаление из датафрейма

Начнем с поверхностного исследования наших данных, чтобы опробовать базовые возможности pandas + выделить интересности, которые может детальнее раскопать далее. Посмотрим, сколько среди наших студентов девушек и парней

In [ ]:
data['sex'].value_counts()

In [ ]:
data.groupby('sex')['sex'].count()

df.groupby() - в целом работает как и GROUPBY() в SQL. Позволяет сгруппировать наболр данных по полю или набору полей и получить соответствующие агрегаты

In [ ]:
data.groupby(['sex','school'])['age'].mean().round()

In [ ]:
data.describe()

Отделим числовые признаки от нечисловых:

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print ('Категориальные признаки: ',categorical_columns)
print ('числовые признаки: ',numerical_columns)

Построим корреляционную матрицу Пирсона:

In [ ]:
cols = data[['age','studytime','Dalc','Walc','G3','absences','Medu','Fedu','health','famrel']]
plt.figure(figsize=(20,10))
c= cols.corr()
sns.heatmap(c,cmap="BrBG",annot=True)

In [ ]:
plt.figure(figsize=(20,10))
c= data[numerical_columns].corr()
sns.heatmap(c,cmap="BrBG",annot=True)

Выделим интересующие нас векторы, не стоит брать много за раз. Лучше концентрироваться на одной детали за раз. Например, начнем с успеваемости студентов и попробуем выделить факторы, влияющие на нее положительно или негативно.

Успеваемость и время, проведенное за учебой

Weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours). Сделаем почитабельнее
Сгруппируем время, которое студенты посвящают обучению с помощью нового столбца:

In [ ]:
data['study_type'] = np.where(data['studytime'] == 1,'<2 hours',
                             np.where(data['studytime'] == 2,'2-5h',
                                     np.where(data['studytime'] == 3,'5-10h','> 10h')))

In [ ]:
#Сгруппируйте объекты датафрейма по новому столбцу. Сколько времени чаще всего посвящают обучению студенты?

In [ ]:
sns.catplot(x="study_type", kind="count",palette="magma", data=data, height = 6)

In [ ]:
g = sns.FacetGrid(data, row="study_type",
                  height=1.7, aspect=5)
g.map(sns.distplot, "G3", hist=True, rug=True, color = 'blue')

In [ ]:
data['G3'].agg({'min','max','mean','median'})

In [ ]:
#Посчитайте те же показатели для оценки первого и второго семестра

Box plot (ящик с усами) - показывает распределение, выделяя медиану, нижний и верхний квартили, минимальное и максимальное значение выборки и выбросы.

In [ ]:
plt.figure(figsize=(18,7))
sns.boxplot(y="study_type", x="G3", data = data , orient="h", palette = 'Set2')

In [ ]:
#Нарисуйте ящик с усами по оценкам за второй семестр

In [ ]:
#пропуски и суммарный балл
g = sns.FacetGrid(data, palette='magma', height=5)
g.map(plt.scatter, "absences", "G3", s=50, alpha=.7, linewidth=.5, edgecolor="white")
g.add_legend()

Получим немного общей инфы в разрезе успеваемости

In [ ]:
data.groupby(['study_type','sex','school'])['G3','absences'].agg({'min','max','median'})

In [ ]:
data['school'].value_counts()

In [ ]:
g = sns.FacetGrid(data, row="school",
                  height=1.7, aspect=5)
g.map(sns.distplot, "G3", hist=True, rug=True, color = 'pink')

In [ ]:
data.groupby('school')['G3'].quantile([.25,.50,.75,.95])

Перейдем к более интересным вещам) Посмотрим как на успеваемость влияют употребление алкоголя, отношения, здоровье, ситуция в семье, место жительства и так далее

In [ ]:
#Создайте здесь

In [ ]:
#Создайте здесь

In [ ]:
labels = data["Weekends_alco"].unique().tolist()
amount = data["Weekends_alco"].value_counts().tolist()

colors = ["pink", "cyan", "green", "yellow"]

trace = go.Pie(labels=labels, values=amount,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
dt = [trace]
layout = go.Layout(title="Weekends alco")

fig = go.Figure(data=dt, layout=layout)
iplot(fig, filename='pie')

In [ ]:
labels = data["Week_alco"].unique().tolist()
amount = data["Week_alco"].value_counts().tolist()

colors = ["cyan", "pink", "white", "yellow"]

trace = go.Pie(labels=labels, values=amount,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
dt = [trace]
layout = go.Layout(title="Week alco")

fig = go.Figure(data=dt, layout=layout)
iplot(fig, filename='pie')

Как на успеваемости и пропусках сказывается любовь к выпивке на выходных?)

In [ ]:
data.groupby('Weekends_alco')['G3','absences'].quantile([.25,.50,.90])

In [ ]:
g = sns.FacetGrid(data, row="Weekends_alco",
                  height=1.7, aspect=5)
g.map(sns.distplot, "G3", hist=True, rug=True, color = 'green')

In [ ]:
g = sns.FacetGrid(data, row="Weekends_alco",
                  height=1.7, aspect=5)
g.map(sns.distplot, "absences", hist=True, rug=True, color = 'red')

Violin plot (скрепичная диаграмма) - используется для визуализации распределения данных и их плотности вероятности. Скрипичный график – это сочетание диаграммы размаха и графика плотности, развернутых и расположенных по обе стороны для отображения формы распределения данных

In [ ]:
plt.figure(figsize=(15,7))
sns.violinplot(x="Weekends_alco", y="G3", data=data, palette = 'rainbow')

In [ ]:
plt.figure(figsize=(15,7))
sns.violinplot(x="Weekends_alco", y="absences", data=data, palette = 'magma')

In [ ]:
data.groupby(['Week_alco'])['G3','absences'].quantile([.25,.50,.75,.95])

In [ ]:
data[(data['Week_alco'] == 'high')]

In [ ]:
always_alco = data[(data['Week_alco'] == 'high') & (data['Weekends_alco'] == 'high')]
never_alco = data[(data['Week_alco'] == 'low') & (data['Weekends_alco'] == 'low')]

In [ ]:
f= plt.figure(figsize=(18,7))

ax=f.add_subplot(121)
sns.distplot(always_alco['G3'],color = 'pink',ax=ax)
ax.set_title(' G3 always alco')

ax=f.add_subplot(122)
sns.distplot(never_alco['G3'],color='gray',ax=ax)
ax.set_title('G3 never alco')

In [ ]:
sns.catplot(x = 'romantic', kind="count",palette="brg", data=data, height = 6)
plt.title("romantic status")

In [ ]:
data.groupby(['school','romantic'])['G3'].agg({'mean','count'})

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(x="romantic", y="G3", data=data, palette = 'magma')

In [ ]:
data.groupby('romantic')['G3','absences'].quantile([.50,.95])

In [ ]:
labels = data["health"].unique().tolist()
amount = data["health"].value_counts().tolist()

colors = ["coral","lightgreen","pink","cyan","white"]

trace = go.Pie(labels=labels, values=amount,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

dt = [trace]
layout = go.Layout(title="Current health status (numeric: from 1 - very bad to 5 - very good)")

fig = go.Figure(data=dt, layout=layout)
iplot(fig, filename = 'h_chart')